In [1]:
import mechanicalsoup
import pandas as pd
import sqlite3

In [2]:
browser = mechanicalsoup.StatefulBrowser()

browser.open("https://fbref.com/en/players/67ac5bb8/matchlogs/2021-2022/Gabriel-Dos-Santos-Match-Logs")

th_elements = browser.page.find_all("th", attrs={"class": 'left'})
filtered_th_elements = [th for th in th_elements if "iz" not in th.get("class", [])]

In [3]:
Date = [value.text for value in filtered_th_elements] # We're only interested in the text not plus tags

In [11]:
Date

['2021-08-27',
 '2021-09-11',
 '2021-09-18',
 '2021-09-26',
 '2021-10-02',
 '2021-10-18',
 '2021-10-22',
 '2021-10-30',
 '2021-11-07',
 '2021-11-11',
 '2021-11-16',
 '2021-11-20',
 '2021-11-27',
 '2021-12-02',
 '2021-12-06',
 '2021-12-11',
 '2021-12-15',
 '2021-12-18',
 '2021-12-21',
 '2021-12-26',
 '2022-01-01',
 '2022-01-13',
 '2022-01-20',
 '2022-01-23',
 '2022-01-27',
 '2022-02-01',
 '2022-02-10',
 '2022-02-19',
 '2022-02-24',
 '2022-03-06',
 '2022-03-13',
 '2022-03-16',
 '2022-03-19',
 '2022-04-04',
 '2022-04-09',
 '2022-04-16',
 '2022-04-20',
 '2022-04-23',
 '2022-05-01',
 '2022-05-08',
 '2022-05-12',
 '2022-05-16',
 '2022-05-22']

In [5]:
td_elements = browser.page.find_all("td", attrs={"class": lambda x: x and 'left iz' not in x and 'left iz group_start' not in x and 'center iz' not in x})
filtered_td_elements = [td for td in td_elements]


In [6]:
from bs4 import BeautifulSoup

allowed_classes = ['center', 'left', 'right', 'right iz', 'right iz group_start', 'left group_start']

td_elements = browser.page.find_all("td", attrs={"class": lambda x: x and any(cls in x for cls in allowed_classes)})
filtered_td_elements = [td for td in td_elements if not td.find_parent("tr", class_="spacer partial_table")]

In [7]:
columns = [value.text for value in filtered_td_elements]

In [8]:
columns

['Fri',
 'PL2 — Div. 1',
 'Matchweek 3',
 'Away',
 'W 2–1',
 'Arsenal',
 'Tottenham',
 'Y*',
 'CB',
 '45',
 '0',
 '0',
 '0',
 '0',
 '',
 '',
 '0',
 '0',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Match Report',
 'Sat',
 'Premier League',
 'Matchweek 4',
 'Home',
 'W 1–0',
 'Arsenal',
 'Norwich City',
 'Y',
 'CB',
 '90',
 '0',
 '0',
 '0',
 '0',
 '2',
 '0',
 '0',
 '0',
 '58',
 '2',
 '0',
 '3',
 '0.1',
 '0.1',
 '0.0',
 '2',
 '0',
 '45',
 '48',
 '93.8',
 '4',
 '40',
 '0',
 '0',
 '0',
 'Match Report',
 'Sat',
 'Premier League',
 'Matchweek 5',
 'Away',
 'W 1–0',
 'Arsenal',
 'Burnley',
 'Y',
 'CB',
 '90',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '70',
 '3',
 '0',
 '2',
 '0.0',
 '0.0',
 '0.0',
 '0',
 '0',
 '45',
 '54',
 '83.3',
 '0',
 '35',
 '0',
 '0',
 '0',
 'Match Report',
 'Sun',
 'Premier League',
 'Matchweek 6',
 'Home',
 'W 3–1',
 'Arsenal',
 'Tottenham',
 'Y',
 'CB',
 '90',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '58',
 '0',

In [9]:
col_names = ['Day',
             'Comp',
             'Round',
             'Venue',
             'Result',
             'Squad',
             'Opponent',
             'Start',
             'Pos',
             'Min',
             'Gls',
             'Ast',
             'PK',
             'PKatt',
             'Sh',
             'SoT',
             'CrdY',
             'CrdR',
             'Touches',
             'Tkl',
             'Int',
             'Blocks',
             'xG',
             'npxG',
             'xAG',
             'SCA',
             'GCA',
             'Cmp',
             'Att',
             'Cmp%',
             'PrgP',
             'Carries',
             'PrgC',
             'Att',
             'Succ',
             'Match Report']

In [10]:
import pandas as pd

col_names = ['Day',
             'Comp',
             'Round',
             'Venue',
             'Result',
             'Squad',
             'Opponent',
             'Start',
             'Pos',
             'Min',
             'Gls',
             'Ast',
             'PK',
             'PKatt',
             'Sh',
             'SoT',
             'CrdY',
             'CrdR',
             'Touches',
             'Tkl',
             'Int',
             'Blocks',
             'xG',
             'npxG',
             'xAG',
             'SCA',
             'GCA',
             'Cmp',
             'Att',
             'Cmp%',
             'PrgP',
             'Carries',
             'PrgC',
             'Att',
             'Succ',
             'Match Report']

consistent_occurrences = []
target_gap = 36  # Assuming you want a consistent gap of 36

# Iterate through the list and find occurrences
for i in range(len(columns)):
    if columns[i] == 'Match Report':
        consistent_occurrences.append(i)

# Create a list to store rows
consistent_rows = []

# Process only the last occurrence in each group
for occurrence in consistent_occurrences:
    start_index = occurrence - target_gap + 1
    if start_index >= 0:
        consistent_rows.append(columns[start_index:occurrence + 1])

# Create a DataFrame with column names
df = pd.DataFrame(consistent_rows, columns=col_names)
df.index = df.index +1
# Display the DataFrame
df.to_csv('File1.csv', encoding='utf-8')


PermissionError: [Errno 13] Permission denied: 'File1.csv'